## Read AEWAS reports

E. Quinn 12/15/2020

This notebook uses pdfminer to extract the information from the RI Municipal Finance AEWAS reports

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf


## Import standard python datascience packages

In [ ]:
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

## Import pdfminer packages

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

### Show the directory we are running in

In [ ]:
!pwd

## Read the pdf and create a dictionary with the contents of each text box

### Function read_pdf() reads a PDF and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object
    

In [ ]:
def read_pdf(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

In [ ]:
def make_num(nstr):
    return(nstr.replace('$','').replace(',','').replace('%',''))

In [ ]:
ewtab = {}
ewtab[2022] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2022_2018-AEWAV-as-of-August-2020.pdf" 
ewtab[2021] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2021_2017-AEWAV-as-of-August-2019.pdf"
ewtab[2020] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2020_2016-AEWAV-as-of-August-30-2018.pdf"
ewtab[2019] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2019_2015_AEWAV-Second-Revision-as-of-8-30-18.pdf"
ewtab[2018] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2018_2014_AEWAV-as-of-July-2016.pdf"
ewtab[2017] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2017_2013_AEWAV-as-of-May-2015.pdf"
ewtab[2016] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2016_10-AEWAV-2012-as-of-August-2014.pdf"
ewtab[2015] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2015_7_AEWAV-as-of-August-2011_2009.pdf"
ewtab[2014] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2014_6_AEWAV-as-of-August-2010_2008.pdf"
ewtab[2013] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2013_5_AEWAV-as-of-August-2009_2007.pdf"
ewtab[2012] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2012_4_AEWAV-as-of-August 2008_2006.pdf"
ewtab[2011] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2011_3_AEWAV-as-of-August-2007_2005.pdf"
ewtab[2010] = "../RI_Div_of_Muncipal_Finance/AEWAV/FY2010_2_AEWAV-as-of-August-2006_2004.pdf"
ewtab[2009] = "../RI_Div_of_Muncipal_Finance/AEWAV/Fy2009_1_AEWAV_2003.pdf"

In [ ]:
ewas = {}

cols = {1:'town',2:'assessed',3:'percent_assessed',4:'full_value',5:'EWAV_unadj', \
        6:'income_adj',7:'adjusted_EWAV',8:'prescribed_ewav'}

colkeys ={}
colkeys[2015] = [20,21,22,23,9,12,17,18]
colkeys[2014] = [22,23,24,25,9,12,15,19]
colkeys[2013] = [22,23,24,25,9,12,15,19]
colkeys[2012] = [22,23,24,25,9,12,15,19]
colkeys[2011] = [11,10,12,15,16,19,24,25]
colkeys[2010] = [9,8,11,10,14,17,20,23]
colkeys[2009] = [9,8,10,11,15,20,21,24]
colkeys[2016] = [11,12,16,17,18,26,28,32]
colkeys[2017] = [11,12,16,17,18,26,28,32]
colkeys[2018] = [11,12,16,17,18,26,28,32]
colkeys[2019] = [12,13,14,15,16,21,23,27]
colkeys[2020] = [11,12,13,14,15,20,22,26]
colkeys[2021] = [12,13,14,15,16,21,23,27]
colkeys[2022] = [6,14,15,16,17,20,26,24]

for fyear in ewtab.keys():
    aew = read_pdf(ewtab[fyear])
    ewas[fyear] = {}
    for i in np.arange(1,1+len(cols)):
        tbox = colkeys[fyear][i-1]
        tlist = aew[0][tbox]['text'].split('\n')
        ewas[fyear][cols[i]] = {}
        for j in np.arange(len(tlist)):
            if (len(tlist[j]) > 0):
                ewas[fyear][cols[i]][j+1] = tlist[j]

ewas        